In [1]:
import os
import pandas as pd
import plotly
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly as pys
import ipywidgets as widgets
from ipywidgets import Layout

import _global_scripts as gs

## Purpose

Consider removing the crt distance variable that discourages close trips and replace with something that discourages close AND encourages far distances

 - **2.7.1** Piecewise function. Up to 60 the slope is half as much, beyond 60 the slop is 1/12 its original pace. 
 - **2.7.2** Run same equation but with only 1 CRT drive access link per TAZ
 

*Please note that E2.7.1 includes a ADJ_CONST_CRT_UT = -5 and ADJ_CONST_CRT_DA =  5 to encourage utah county CRT and discourage davis county crt. I meant to turn these off, but forgot. E2.7.2 sets these back to 0*

## Inputs

In [2]:
# set whether you want PA or OD analysis
pa_od = 'PA'
pa_od_function = 'pa'

In [3]:
df_tdm_obs = pd.read_csv(f"_data/base_observed_summary_{pa_od_function}.csv").sort_values(by=['Source','station'], ascending=True).reset_index(drop=True)
if(pa_od == 'PA'):
        df_tdm_obs = df_tdm_obs.drop(columns={'Alt_PA','Alt_Direct_PA','Alt_Transfer_PA'})

# read in results for e2.7.1 before recalibration
path_brding_summary_271_b4 = r"_data/E2.7/E2.7.1_node_b4calib.csv"
path_rider_summary_271_b4  = r"_data/E2.7/E2.7.1_link_b4calib.csv"

# read in results for e2.7.1 after recalibration
path_brding_summary_271 = r"_data/E2.7/E2.7.1_node.csv"
path_rider_summary_271  = r"_data/E2.7/E2.7.1_link.csv"

# read in results for e2.7.2 after recalibration
path_brding_summary_272 = r"_data/E2.7/E2.7.2_node.csv"
path_rider_summary_272  = r"_data/E2.7/E2.7.2_link.csv"

In [4]:
# read in stations and summarize tdm results
df_stations1 = gs.df_stations[['station','N']]
df_tdm_271_b4 = gs.summarize_tdm_stats(path_brding_summary_271_b4,path_rider_summary_271_b4, df_stations1, 'TDM_2.7.1_b4', pa_od_function)
df_tdm_271    = gs.summarize_tdm_stats(path_brding_summary_271,path_rider_summary_271, df_stations1, 'TDM_2.7.1', pa_od_function)
df_tdm_272    = gs.summarize_tdm_stats(path_brding_summary_272,path_rider_summary_272, df_stations1, 'TDM_2.7.2', pa_od_function)

# concat dataframes
df_tdm_obs_new = pd.concat([
        df_tdm_obs,
        #df_tdm_271_b4,
        df_tdm_271,
        df_tdm_272
    ]).reset_index()
df_tdm_obs_new = df_tdm_obs_new.round(3).fillna(0)
df_tdm_obs_new = df_tdm_obs_new.drop(columns='index')
df_tdm_obs_new = df_tdm_obs_new.loc[:, ~df_tdm_obs_new.columns.str.contains('^Unnamed')]

# rename
sumStats = df_tdm_obs_new.copy()
display(df_tdm_obs_new)

d:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rider_summary = pd.read_csv(path_riders)
d:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rider_summary = pd.read_csv(path_riders)


,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00
...,...,...,...,...,...,...
115,TDM_2.7.2,11-FARMINGTON STATION,walk,189.730,134.91,54.82
116,TDM_2.7.2,12-LAYTON STATION,walk,365.950,140.89,225.06
117,TDM_2.7.2,13-CLEARFIELD STATION,walk,629.680,489.82,139.86
118,TDM_2.7.2,14-ROY STATION,walk,384.810,346.34,38.47


In [5]:
# add a few more columns regarding percentage of boardings in relation to total boardings

# sum by source and station 
station_sum = sumStats.groupby(["Source", "station"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

# add All accessMode
station_sum["AccessMode"] = "All"
sumStats2 = pd.concat([sumStats, station_sum], ignore_index=True) 

accessmode_sum = sumStats2.groupby(["Source", "AccessMode"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

accessmode_sum.rename(columns={
    f"Brd_{pa_od}": f"Source_Brd_{pa_od}", 
    f"Brd_Direct_{pa_od}": f"Source_Brd_Direct_{pa_od}", 
    f"Brd_Transfer_{pa_od}": f"Source_Brd_Transfer_{pa_od}"}, inplace=True)
sumStatsP = sumStats2.merge(accessmode_sum, on=["Source", "AccessMode"], how="left")

sumStatsP[f"Brd_{pa_od}_Perc"]          = sumStatsP[f"Brd_{pa_od}"] / sumStatsP[f"Source_Brd_{pa_od}"]
sumStatsP[f"Brd_Direct_{pa_od}_Perc"]   = sumStatsP[f"Brd_Direct_{pa_od}"] / sumStatsP[f"Source_Brd_Direct_{pa_od}"]
sumStatsP[f"Brd_Transfer_{pa_od}_Perc"] = sumStatsP[f"Brd_Transfer_{pa_od}"] / sumStatsP[f"Source_Brd_Transfer_{pa_od}"]
sumStatsP


,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA,Source_Brd_PA,Source_Brd_Direct_PA,Source_Brd_Transfer_PA,Brd_PA_Perc,Brd_Direct_PA_Perc,Brd_Transfer_PA_Perc
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00,14955.118,0.0,0.00,0.152682,NaN,NaN
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00,5672.892,0.0,0.00,0.166742,NaN,NaN
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00,14955.118,0.0,0.00,0.098630,NaN,NaN
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00,5672.892,0.0,0.00,0.065540,NaN,NaN
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00,14955.118,0.0,0.00,0.095203,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
175,TDM_2.7.2,11-FARMINGTON STATION,All,517.130,462.31,54.82,19696.890,16060.7,3636.19,0.026254,0.028785,0.015076
176,TDM_2.7.2,12-LAYTON STATION,All,1437.730,1212.66,225.07,19696.890,16060.7,3636.19,0.072993,0.075505,0.061897
177,TDM_2.7.2,13-CLEARFIELD STATION,All,1658.550,1518.68,139.87,19696.890,16060.7,3636.19,0.084204,0.094559,0.038466
178,TDM_2.7.2,14-ROY STATION,All,1610.920,1572.39,38.53,19696.890,16060.7,3636.19,0.081786,0.097903,0.010596


In [6]:
def plotit(variable, access_mode):
    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:
    
        filtered_data = sumStatsP[sumStatsP['AccessMode'] == access_mode]
            
        # Create histogram
        fig = px.histogram(
            filtered_data, 
            x="station", 
            y=variable, 
            text_auto='.2s',
            color='Source', 
            barmode='group',
            height=400
        )
        fig.update_layout(
            xaxis_title="Station Name",
            yaxis_title=str(variable),
            legend_title="Model Version"
        )
        
        # Display the plot
        fig.show()
    
    else:
        firstTime = True

In [7]:
lstValues = list([
    f'Brd_{pa_od}',
    f'Brd_Direct_{pa_od}',
    f'Brd_Transfer_{pa_od}',
    f'Brd_{pa_od}_Perc',
    f'Brd_Direct_{pa_od}_Perc',
    f'Brd_Transfer_{pa_od}_Perc'])
accessModeOptions = ['drive', 'walk', 'All']

selectValues = widgets.Select(options=lstValues, value=(f'Brd_{pa_od}' ), description = 'Select Variable')
selectAccessMode = widgets.Dropdown(options=accessModeOptions, value='All', description='Access Mode')

# Set up a global variable to track whether the widgets have been changed
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectValues, selectAccessMode])

# create interactive widget
interactive_output = widgets.interactive_output(plotit, {'variable':selectValues, 'access_mode': selectAccessMode})

display(hbox)
display(interactive_output)
display(output)

Output()

Output()

## Conclusions

- by updating the IVT utility equation, drive boardings increase in Provo, Orem, and throughout Davis County -- this makes sense as these stations include longer trips into CBD region
- the updated IVT equation doesn't bypass the "driving past end of line station" issue, at least not in provo. Orem and Roy saw a large increase in dCRT boardings. Ogden also benefited greatly, but provo only benefited slightly.
- the combination of IVT equation change and limiting dCRT to 1 access link showed promising results. It produced a pattern similar to observed with higher boardings at the end of the line and lower boardings in the CBD region for dCRT trips. 
- overall we conclude that IVT and drive access play large roles in determining which station to board and how many boardings occur